In [1]:
import pandas as pd
import numpy as np
    
# funciones desarrolladas
from functions.cargar_data import *

In [3]:
variables = ['DPTO', 'PERMI07', 'PERMI07_2']

In [4]:
censo = cargar_censo_vars(variables)

In [7]:
def conteo_migrantes(df):
    "Conteo de migrante interno por depto."
    lista_vars = ['total', 'cat_a', 'cat_b', 'cat_c']

    df[lista_vars[0]] = 1
    [df.insert(len(df.columns), lista_vars[i], 0) for i in range(1,4)]
    # destino Mvdeo.
    df.loc[(df.PERMI07 == 3) & (df.DPTO==1), lista_vars[1]] = 1
    # destino interior
    df.loc[(df.PERMI07 == 3) & (df.PERMI07_2== 1), lista_vars[2]] = 1
    # entre interior
    df.loc[(df.PERMI07 == 3) & (df.PERMI07_2!=1) & (df.DPTO!=1) & (df.PERMI07_2.isnull()==False), lista_vars[3]] = 1

    # agrupa
    df_group = df.groupby('DPTO').agg(
                {   'total': sum,
                     lista_vars[1]: sum,
                     lista_vars[2]: sum,
                     lista_vars[3]: sum
                })

    df_group['total_mig'] = df_group[lista_vars[1]] + df_group[lista_vars[2]] + df_group[lista_vars[3]]
    
    return df_group

In [8]:
migrantes = conteo_migrantes(censo)
migrantes

,total,cat_a,cat_b,cat_c,total_mig
DPTO,,,,,
1,1318755,42444,0,0,42444
2,73377,0,914,896,1810
3,520173,0,33127,8286,41413
4,84698,0,1387,1614,3001
5,123203,0,2100,3362,5462
6,57084,0,982,1589,2571
7,25050,0,378,933,1311
8,67047,0,1026,2257,3283
9,58815,0,825,1760,2585


In [47]:
# Exporta CSV
migrantes.to_csv('tablas/migrantes_por_depto.csv')

In [48]:
# exporta CSVT
campos = 'Integer','Integer','Integer','Integer','Integer','Integer'

def export_csvt(lista, filename):
    "Exporta lista a archivo csvt para usars con QGIS"
    campos_str = ','.join([str(elem) for elem in campos])
    with open(filename, 'w') as f:
        f.write(str(campos_str) + "\n")

In [49]:
export_csvt(campos, 'tablas/migrantes_por_depto.csvt')